In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import jax
import subprocess
import re


def shmoogle_smi():
    jax.profiler.save_device_memory_profile("memory.prof")
    pprof_path = "/usr/local/go/pkg/tool/linux_amd64/pprof"
    out = subprocess.run([pprof_path, "-top", "memory.prof"], stdout=subprocess.PIPE)
    stdout = out.stdout.decode("utf-8")
    re_sult = re.search(
        r"Showing nodes accounting for (\d+(?:\.\d+)?)([MG]B)?, (\d+(?:\.\d+)?)% of (\d+(?:\.\d+)?)([MG]B)? total",
        stdout,
    )
    multiplier = 1 / 1000 if re_sult.group(5) == "MB" else 1
    total_mem_usage = float(re_sult.group(4))
    print(
        "Total mem usage:",
        total_mem_usage * multiplier,
        "GB",
        "out of",
        16 * len(jax.devices()),
        "GB",
    )


shmoogle_smi()

Total mem usage: 0.0 GB out of 128 GB


In [3]:
from tokenizer import Tokenizer
from llama2_model import LLaMA

import equinox as eqx
import re
from safetensors import safe_open
from jax.sharding import Mesh, NamedSharding, PartitionSpec
import jax.numpy as jnp
import transformers
import numpy as np
import jax
import jmp

/home/neverix/.pyenv/versions/3.12.0/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
tokenizer = Tokenizer("models/Llama-2-7b-hf/tokenizer.model")
input_ids = tokenizer.encode("Hello world!", bos=True, eos=False)
input_ids

[1, 15043, 3186, 29991]

In [5]:
num_devices = len(jax.devices())
mesh = Mesh(np.array(jax.devices()).reshape(-1, 4), axis_names=("dp", "mp"))
policy = jmp.get_policy("p=bf16,c=bf16,o=bf16")
print("Creating LLaMA...")
llama = LLaMA(mesh, policy=policy)
print("Created LLaMA.")
shmoogle_smi()

Creating LLaMA...
Created LLaMA.
Total mem usage: 29.99 GB out of 128 GB


Main binary filename not available.


In [6]:
print("Loading model...")
print()
for filename in [
    "models/Llama-2-7b-hf/model-00001-of-00002.safetensors",
    "models/Llama-2-7b-hf/model-00002-of-00002.safetensors",
]:
    with safe_open(
        filename,
        framework="numpy",
        device="cpu",
    ) as f:
        for k in f.keys():
            weight = f.get_tensor(k)
            if (
                k.endswith(".weight")
                and not k.endswith("embed_tokens.weight")
                and not k.endswith("norm.weight")
                # and not k.endswith("lm_head.weight")
            ):
                weight = weight.T
            re_sult = re.search(r"layers\.([0-9]+)", k)
            try:
                k = (
                    k[: re_sult.span()[0]]
                    + f"layers[{re_sult.group(1)}]"
                    + k[re_sult.span()[1] :]
                )
            except AttributeError:
                pass
            print("\r" + " " * 80, end="")
            print("\rLoading", k, end="")
            og = eval(f"llama.{k}")
            weight = jax.device_put(weight.astype(og.dtype), device=og.sharding)
            llama = eval(f"eqx.tree_at(lambda l: l.{k}, llama, weight)")
print()
shmoogle_smi()

Loading model...

Loading model.norm.weight                                                       
Total mem usage: 29.99 GB out of 128 GB


Main binary filename not available.


In [27]:
from state_util import extract_arg, statify, dummy_caching, dummy_stateful
from functools import partial


extract_state = partial(extract_arg, arg_name="state", index=-2)
extract_cache = partial(extract_arg, arg_name="cache", index=-1)


# try:
#     jax.tree_util.register_pytree_node(
#         jmp.Policy,
#         lambda policy: (
#             vars(policy),
#             None,
#         ),
#         lambda _, dct: jmp.Policy(**dct),
#     )
# except ValueError:
#     pass
try:
    jax.tree_util.register_pytree_node(
        jax._src.numpy.lax_numpy._ScalarMeta, lambda x: (tuple(), x), lambda _, x: x
    )
except ValueError:
    pass


def tree_multiflat(tree):
    if isinstance(tree, (int, float, bool, str, np.ndarray, jax.Array, jmp.Policy)):
        return []
    leaves, treedef = jax.tree_flatten(tree, is_leaf=lambda x: x is not tree)
    if not leaves:
        return [tree]
    if any(x is tree for x in leaves):
        return
    leaves = [tree_multiflat(leaf) for leaf in leaves]
    return [tree] + leaves


# short for multi-level map
def tree_mlm(fn, tree):
    if not isinstance(tree, eqx.Module):
        return tree
    leaves, treedef = jax.tree_util.tree_flatten_with_path(
        tree, is_leaf=lambda x: x is not tree
    )
    if not leaves:
        return tree
    if any(x is tree for x in leaves):
        return
    leaves = [fn(path, tree_mlm(fn, leaf)) for path, leaf in leaves]
    return jax.tree_util.tree_unflatten(treedef, leaves)


class DebugWrapper(eqx.Module):
    module: eqx.Module
    path: str

    def __init__(self, module, path):
        self.module = module
        self.path = path

    def __call__(self, *args, **kwargs):
        _, _, cache = extract_cache(args, kwargs)
        *output, cache = self.module(*args, **kwargs)
        cache = {**cache, self.path: (args, kwargs, output)}
        return tuple(output) + (cache,)

    def __hasattr__(self, attr):
        return hasattr(self.module, attr)

    def __getattr__(self, attr):
        return getattr(self.module, attr)


def debugify_model(model_base):
    def debugify(path, leaf):
        if not isinstance(leaf, eqx.Module):
            return leaf
        if not hasattr(leaf, "__call__"):
            return leaf

        path = "".join(map(str, path))
        return DebugWrapper(leaf, path)

    model = tree_mlm(debugify, model_base)
    model, state_base = statify(model)
    
    def new_model(*args, **kwargs):
        output, state, cache = model(*args, **{**kwargs, "state": state_base, "cache": {}})

        debugs = [x for x in tree_multiflat(model) if isinstance(x, DebugWrapper)]
        debug = {db.path: state.get(db.index) for db in debugs}

        return output, debug

    return new_model


llama_debug = jax.vmap(debugify_model(llama))
shmoogle_smi()

with mesh:
    input_ids = [
        tokenizer.encode(x, bos=True, eos=False)
        for x in ["Hello world", "This is a test"]
    ]
    input_ids = [x + [0] * (128 - len(x)) for x in input_ids]
    ids = jnp.asarray(input_ids)
    ids = jax.device_put(ids, NamedSharding(mesh, spec=PartitionSpec("dp", None)))
    result = llama_debug(ids)

Main binary filename not available.


Total mem usage: 30.29 GB out of 128 GB


/tmp/ipykernel_27789/1798389964.py:31: DeprecationWarning: jax.tree_flatten is deprecated: use jax.tree_util.tree_flatten.
  leaves, treedef = jax.tree_flatten(tree, is_leaf=lambda x: x is not tree)


In [ ]:
reference_llama = transformers.LlamaModel.from_pretrained("models/Llama-2-7b-hf")

In [ ]:
import torch


def make_torch_hook(name):
    def torch_hook(module, input, output):
        print(f"Module called: {name} {module.__class__}")
        for arg in input:
            if isinstance(arg, torch.Tensor):
                print("Input:", arg.shape, arg.dtype, str(arg)[:100])
        if isinstance(output, tuple):
            output = output[0]
        if isinstance(output, torch.Tensor):
            print("Output:", output.shape, output.dtype, str(output)[:100])
        return output

    return torch_hook


for name, module in reference_llama.named_modules():
    module._forward_hooks.clear()
    module.register_forward_hook(make_torch_hook(name))


input_ids = torch.tensor(input_ids)
reference_llama(input_ids)